# This will run training on the training set, and evaluate against validation

This is directed for stage 2 training. For stage 1 training, pipeline.config and file structures/paths will have to be corrected. 

# mount drive

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# clone tensorflow models directory, install required dependencies

In [0]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!pip install pascal-voc-writer
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools

Cloning into 'models'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 33869 (delta 24), reused 54 (delta 7), pack-reused 33794
Receiving objects: 100% (33869/33869), 512.30 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (21817/21817), done.
Checking out files: 100% (2491/2491), done.
Selecting previously unselected package python-bs4.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing 

In [0]:
%tensorflow_version 1.x

from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

import sys
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')

In [0]:
print(tf.__version__)

1.15.2


# build and setup, set path

In [0]:
!python3 /content/models/research/setup.py build
!python3 /content/models/research/setup.py install

os.environ['PYTHONPATH'] = '/tensorflow-1.15.2/python3.6:/env/python/:/content/models/research:/content/models/research/slim:/content/models/research/object_detection'

print(os.environ['PYTHONPATH'])

running build
running install
running bdist_egg
running egg_info
creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/e

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 5.68 MiB/s, done.
Resolving deltas: 100% (576/576), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/

# build .proto files

In [0]:
!cd /content/models/research;protoc object_detection/protos/*.proto --python_out=.

# check object_detection installed correctly

In [19]:
!python3 /content/models/research/object_detection/builders/model_builder_test.py

#output at the end should be OK

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

In [0]:
print(len(os.listdir('/content/drive/My Drive/all_img_final')))
#print(len(files))
print(len(os.listdir('/content/drive/My Drive/xmls/testxml')))
print(len(os.listdir('/content/drive/My Drive/xmls/trainxml')))
print(len(os.listdir('/content/drive/My Drive/xmls/valxml')))

16508
2004
12500
2004


# to create tfrecord

In [0]:
'''from object_detection.utils import dataset_util
#run once

#change this to the base directory where your data/ is 
data_base_url = '/content/drive/My Drive'

#location of images
image_dir = os.path.join(data_base_url,'all_img_final')

def class_text_to_int(row_label):
    if row_label == 'glioma':
        return 1
    elif row_label == 'meningioma':
        return 2
    elif row_label == 'pituitary tumour':
        return 3
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
		    xmins.append(row['xmin'] / width)
		    xmaxs.append(row['xmax'] / width)
		    ymins.append(row['ymin'] / height)
		    ymaxs.append(row['ymax'] / height)
		    classes_text.append(row['class'].encode('utf8'))
		    classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
		    'image/height': dataset_util.int64_feature(height),
		    'image/width': dataset_util.int64_feature(width),
		    'image/filename': dataset_util.bytes_feature(filename),
		    'image/source_id': dataset_util.bytes_feature(filename),
		    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		    'image/format': dataset_util.bytes_feature(image_format),
		    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		    'image/object/class/label': dataset_util.int64_list_feature(classes),
		    }))

    return tf_example

#creates tfrecord for csvs
for csv in ['xmls/trainxml']:
    idx = 1
    writer = tf.io.TFRecordWriter(os.path.join(data_base_url, csv)+'.record')
    path = os.path.join(image_dir)
    examples = pd.read_csv(os.path.join(data_base_url,csv) + '.csv')
    grouped = split(examples, 'filename')
    for group in grouped:
        print(idx)
        idx += 1
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    
    writer.close()
    output_path = os.path.join(data_base_url,csv) + '.record'
    print('Successfully created the TFRecords: {}'.format(os.path.join(data_base_url,csv) + '.record'))'''

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: ignored

# select model and download

if resuming training, do not need to run the next 3 cells.
if fine tuning pre-trained model, do not need to run the next  cells. 

In [0]:
# Some models to train on
MODELS_CONFIG = {
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
    'faster_rcnn_resnet50': {
        'model_name': 'faster_rcnn_resnet50_coco_2018_01_28',
    },
    'faster_rcnn_resnet101': {
        'model_name': 'faster_rcnn_resnet101_coco_2018_01_28',
    },
    'faster_rcnn_inception_resnet_v2': {
        'model_name': 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28',
    },
}

# Select a model from `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'faster_rcnn_inception_resnet_v2'

In [0]:
#the distination folder where the model will be saved
#change this if you have a different working dir
DEST_DIR = '/content/drive/My Drive/model_frcnn_inception_resnet_v2_final'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
shutil.move(MODEL, DEST_DIR)

"# Name of the object detection model to use.\nMODEL = MODELS_CONFIG[selected_model]['model_name']\n\n#selecting the model\nMODEL_FILE = MODEL + '.tar.gz'\n\n#creating the downlaod link for the model selected\nDOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'\n\n#checks if the model has already been downloaded, download it otherwise\nif not (os.path.exists(MODEL_FILE)):\n    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)\n\n#unzipping the model and extracting its content\ntar = tarfile.open(MODEL_FILE)\ntar.extractall()\ntar.close()\n\n# creating an output file to save the model while training\nos.remove(MODEL_FILE)\nif (os.path.exists(DEST_DIR)):\n    shutil.rmtree(DEST_DIR)\nshutil.move(MODEL, DEST_DIR)"

In [0]:
# if above cell was not run, manually set DEST_DIR
CONFIG_PATH = DEST_DIR

In [21]:
#check pipeline.config correct
#!cat /content/drive/'My Drive'/model_frcnn_inception_resnet_v2_final/pipeline.config



model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_resnet_v2'
      first_stage_features_stride: 8
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 8
        width_stride: 8
      }
    }
    first_stage_atrous_rate: 2
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 

# write pipeline.config if necessary

In [0]:
%%writefile pipeline.config


model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_resnet_v2'
      first_stage_features_stride: 8
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 8
        width_stride: 8
      }
    }
    first_stage_atrous_rate: 2
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 17
    maxpool_kernel_size: 1
    maxpool_stride: 1
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0001
          schedule {
            step: 50000
            learning_rate: 0.00009
          }
          schedule {
            step: 100000
            learning_rate: 0.000081
          }
          schedule {
            step: 150000
            learning_rate: 0.0000729
          }
          schedule {
            step: 200000
            learning_rate: 0.00006561
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/drive/My Drive/model_frcnn_inception_resnet_v2_final/model.ckpt"
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 250000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/xmls/trainxml.record"
  }
  label_map_path: "/content/drive/My Drive/label_map.pbtxt"
}

eval_config: {
  num_examples: 2004
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 2004
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/xmls/valxml.record"
  }
  label_map_path: "/content/drive/My Drive/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Writing pipeline.config


In [0]:
!rm /content/drive/'My Drive'/model_frcnn_inception_resnet_v2_final/pipeline.config
shutil.move('pipeline.config', CONFIG_PATH)

'/content/drive/My Drive/model_frcnn_inception_resnet_v2_final/pipeline.config'

In [22]:
#check model_main.py is correct
#!cat /content/models/research/object_detection/model_main.py


# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Binary to run train and evaluation on object detection model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags

import tensorflow as tf

from object_detection import model_hparams
from object_detection import model_lib


# write model_main.py if necessary

In [0]:
%%writefile /content/models/research/object_detection/model_main.py

# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Binary to run train and evaluation on object detection model."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags

import tensorflow as tf

from object_detection import model_hparams
from object_detection import model_lib

flags.DEFINE_string(
    'model_dir', None, 'Path to output model directory '
    'where event and checkpoint files will be written.')
flags.DEFINE_string('pipeline_config_path', None, 'Path to pipeline config '
                    'file.')
flags.DEFINE_integer('num_train_steps', None, 'Number of train steps.')
flags.DEFINE_boolean('eval_training_data', False,
                     'If training data should be evaluated for this job. Note '
                     'that one call only use this in eval-only mode, and '
                     '`checkpoint_dir` must be supplied.')
flags.DEFINE_integer('sample_1_of_n_eval_examples', 1, 'Will sample one of '
                     'every n eval input examples, where n is provided.')
flags.DEFINE_integer('sample_1_of_n_eval_on_train_examples', 5, 'Will sample '
                     'one of every n train input examples for evaluation, '
                     'where n is provided. This is only used if '
                     '`eval_training_data` is True.')
flags.DEFINE_string(
    'hparams_overrides', None, 'Hyperparameter overrides, '
    'represented as a string containing comma-separated '
    'hparam_name=value pairs.')
flags.DEFINE_string(
    'checkpoint_dir', None, 'Path to directory holding a checkpoint.  If '
    '`checkpoint_dir` is provided, this binary operates in eval-only mode, '
    'writing resulting metrics to `model_dir`.')
flags.DEFINE_boolean(
    'run_once', False, 'If running in eval-only mode, whether to run just '
    'one round of eval vs running continuously (default).'
)
FLAGS = flags.FLAGS


def main(unused_argv):
  flags.mark_flag_as_required('model_dir')
  flags.mark_flag_as_required('pipeline_config_path')
  config = tf.estimator.RunConfig(model_dir=FLAGS.model_dir, save_checkpoints_steps=12500, keep_checkpoint_max=200)

  train_and_eval_dict = model_lib.create_estimator_and_inputs(
      run_config=config,
      hparams=model_hparams.create_hparams(FLAGS.hparams_overrides),
      pipeline_config_path=FLAGS.pipeline_config_path,
      train_steps=FLAGS.num_train_steps,
      sample_1_of_n_eval_examples=FLAGS.sample_1_of_n_eval_examples,
      sample_1_of_n_eval_on_train_examples=(
          FLAGS.sample_1_of_n_eval_on_train_examples))
  estimator = train_and_eval_dict['estimator']
  train_input_fn = train_and_eval_dict['train_input_fn']
  eval_input_fns = train_and_eval_dict['eval_input_fns']
  eval_on_train_input_fn = train_and_eval_dict['eval_on_train_input_fn']
  predict_input_fn = train_and_eval_dict['predict_input_fn']
  train_steps = train_and_eval_dict['train_steps']

  if FLAGS.checkpoint_dir:
    if FLAGS.eval_training_data:
      name = 'training_data'
      input_fn = eval_on_train_input_fn
    else:
      name = 'validation_data'
      # The first eval input will be evaluated.
      input_fn = eval_input_fns[0]
    if FLAGS.run_once:
      estimator.evaluate(input_fn,
                         steps=None,
                         checkpoint_path=tf.train.latest_checkpoint(
                             FLAGS.checkpoint_dir))
    else:
      model_lib.continuous_eval(estimator, FLAGS.checkpoint_dir, input_fn,
                                train_steps, name)
  else:
    train_spec, eval_specs = model_lib.create_train_and_eval_specs(
        train_input_fn,
        eval_input_fns,
        eval_on_train_input_fn,
        predict_input_fn,
        train_steps,
        eval_on_train_data=False)

    # Currently only a single Eval Spec is allowed.
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_specs[0])


if __name__ == '__main__':
  tf.app.run()

Overwriting /content/models/research/object_detection/model_main.py


# run to start training

ensure pipeline_config_path and model_dir are correct

In [0]:
!python3 /content/models/research/object_detection/model_main.py \
    --pipeline_config_path=/content/drive/My\ Drive/model_frcnn_inception_resnet_v2_final/pipeline.config \
    --model_dir=/content/drive/My\ Drive/model_frcnn_inception_resnet_v2_final/training/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0414 09:39:57.920130 140296329365376 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0414 09:39:57.923847 140296329365376 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0414 09:39:57.924021 140296329365376 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0414 09:39:57.924160 140296329365376 module_wrapper.py:139] From /content/models/resea

In [0]:
#use this for renaming if necessary
#!mv /content/drive/'My Drive'/model_frcnn_inception_resnet_v2_final/'model (1).ckpt.index' /content/drive/'My Drive'/model_frcnn_inception_resnet_v2_final/model.ckpt.index

In [0]:
#additional code for rpn2 and more can be found in frozen_graph.ipynb and actual_inference.ipynb
#not provided here for simplicity